In [1]:
import pandas as pd
import string 
from nltk.corpus import stopwords

In [2]:
#Get the spam data collection 
df_spamCollection = pd.read_csv('SpamCollection', sep='\t', names=['response','message'])

In [3]:
df_spamCollection.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:

df_spamCollection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [5]:
#view response 

In [6]:
#Verify length of the messages and also add it as a new column 
df_spamCollection['length'] = df_spamCollection['message'].apply(len)

In [7]:
df_spamCollection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [8]:
#define a function to get rid of stopwords present in the messages
def message_text_process(mess):
    no_puntuation = [char for char in mess if char not in string.punctuation]
    no_puntuation = ''.join(no_puntuation)
    return[word for word in no_puntuation.split() if word.lower() not in stopwords.words('english')]
    

In [9]:
df_spamCollection['message'].head(5).apply(message_text_process)


0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [10]:
#start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
#use bag of words by applying the function and fit the data into it
bag_of_words_transformer = CountVectorizer(analyzer=message_text_process).fit(df_spamCollection['message'])

In [12]:
#print length of bag of words stored in the vocabulary_ attribute
print(len(bag_of_words_transformer.vocabulary_))

11425


In [13]:
message_bagofwords = bag_of_words_transformer.transform(df_spamCollection['message'])


In [14]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfid_transformer = TfidfTransformer().fit(message_bagofwords)

In [15]:
#print shape of the tfidf 
message_tfid = tfid_transformer.transform(message_bagofwords)
print(message_tfid.shape)

(5572, 11425)


In [16]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(message_tfid,df_spamCollection['response'])

In [17]:
#check model for the predicted and expected value say for message#2 and message#5
message = df_spamCollection['message'][2]
bag_of_words_for_message = bag_of_words_transformer.transform([message])
tfid = tfid_transformer.transform(bag_of_words_for_message)

print('predicted', spam_detect_model.predict(tfid)[0])
print('expected', df_spamCollection.response[2])

predicted spam
expected spam
